![picture](https://avatars.githubusercontent.com/u/55989987?s=200&v=4)
## **[PRATICA]** Introdução à PLN na área da saúde
Bem-vindo(a) ao tutorial prático da aula  **Introdução à PLN na área da saúde**. Vamos ver algumas tarefas de processamento de texto, reconhecimento de entidades e testar um modelo Word2vec treinado pelo nosso grupo de pesquisa, [HAILab](https://github.com/HAILab-PUCPR/).

Temas abordados:

1.  Baixando um corpus clínico
2.  Processamento básico de textos com o pacote [NLTK](https://www.nltk.org/)
3.  Reconhecimento de entidades nomeadas (NER)
4.  Explorando um modelo Word2vec treinado com *tweets* brasileiros sobre Covid-19

### **Relembrando: O que é Processamento de Linguagem Natural?**

É uma sub-área da Inteligência Artificial que ajuda computadores a entender, interpretar e manipular a linguagem natural (linguagem escrita ou falada)

O PLN é uma área de fundamental importância na extração informação de textos clínicos.










### **1) Baixando um corpus clínico**


Vamos utilizar um dataset de textos biomédicos em Português Europeu, disponível [neste link](https://github.com/fabioacl/PortugueseClinicalNER).

> **OBSERVAÇÃO**: O corpus contendo narrativas clínicas anotadas para NER, [SemClinBr](https://github.com/HAILab-PUCPR/SemClinBr), desenvolvido pelo HAILab, está disponível para acesso mediante solicitação, para fins acadêmicos e de pesquisa.



In [ ]:
# Instala pacote gitPython, que serve para copiar arquivos do GitHub
!pip install gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00


In [ ]:
import git
# Copia repositório para diretório atual
git.Git("./").clone("https://github.com/fabioacl/PortugueseClinicalNER.git")

''

In [ ]:
!ls PortugueseClinicalNER

 README.md		       'Texts SPN 2 Raw'
'Texts SPN 1 Labeled English'  'Texts Word Embeddings Training'
'Texts SPN 1 Raw'	       'Word Embedding Model'
'Texts SPN 2 Labeled English'


Agora vamos armazenar alguns dos dados dos arquivos originais, em uma lista.

In [ ]:
import os

# Cria lista
textos = []
directory = "PortugueseClinicalNER/Texts SPN 2 Raw/"

# Percorre arquivos do diretório
for filename in os.listdir(directory):
  #Se é um arquivo texto
  if filename.endswith(".txt"):

    # Abre arquivo
    f = open(os.path.join(directory, filename), "r", encoding='cp1252')
    # Adiciona texto na lista
    textos.append(f.read().replace("\n", " "))
    # Guarda

  else:
    continue

In [ ]:
textos[:2]

['Adolescente de 13 anos com DW com confirmação genética e apresentação neurológica por doença de movimento (parkinsonismo e distonia) com agravamento progressivo, sem evidência clínica ou analítica de atingimento hepático. A biópsia hepática apresentava infiltrado inflamatório e sinais de hepatopatia crónica, sem cirrose. A RMN cerebral mostrou hipersinal T2 nos gânglios da base (caudado e putámen), sem alterações em T1 e restrição à difusão no caudado, núcleo lenticular e mesencéfalo. O início do tratamento com penicilamina despoletou uma progressiva deterioração neurológica sem estabilização após 11 meses de follow-up.',
 'Homem de 65 anos, com antecedentes de diabetes mellitus tipo 2 controlada, com quadro de 3 meses de evolução caracterizado por tosse não produtiva, mialgias, febre, anorexia e perda de peso. Apresenta posteriormente agravamento com quadro de confusão mental com 4 dias de evolução, razão pela qual recorre ao Serviço de Urgência. À observação, febril (38.5ºC), lenti



### **2) Processando textos com o pacote NLTK**


O NLTK (*Natural Language Toolkit*) é uma biblioteca para processamento de linguagem natural que fornece várias funcionalidades de PLN.

O NLTK já vem instalado no ambiente do *Google Colab* por padrão, porém, alguns recursos como corpora textuais podem não estar disponíveis. Para tal, você deve explicitamente recomendar seu *download* usando o comando a seguir:

```nltk.download()```

> **OBSERVAÇÃO**: Você pode optar por utilizar outros pacotes como o [Stanza](https://stanfordnlp.github.io/stanza/) e o [spaCy](https://spacy.io/).



In [ ]:
# Você deve importar o tokenizador da biblioteca NLTK
import nltk
from nltk import tokenize

# Caso não tenha feito o download de todos recursos do NLTK, você pode fazê-lo de maneira individual
# aqui vamos obter os recursos relacionados à tokenização (modelos pré-treinados, regras gramaticais e outros recursos)
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### Operações Básicas ou Pré-processamento
As operações a seguir geralmente fazem parte de uma etapa do PLN chamada de **Pré-processamento**, e já se tornaram triviais a todos programas de PLN, pois preparam o texto bruto para ser realmente processado e "entendido" pela máquina.

*   Segmentação de sentenças
*   Tokenização
*   Normalização
*   *Stemming*
*   Lematização



##### Segmentação de sentenças
As regras principais de segmentação de sentenças contam com a divisão a partir de pontuações encontradas no texto ou quebras de linha.

![Exemplo de segmentação de sentença - Dan Jurafsky](https://drive.google.com/uc?export=view&id=15fLctvbOi8_STzO8AOzzW095yA2IvPmK)

In [ ]:
# Exemplo de tokenização a nivel de sentença
# Vamos pegar as sentencas do nosso texto, usando a função 'sent_tokenize'

from nltk import sent_tokenize

sents = sent_tokenize(textos[0])
# imprime as 5 primeiras frases
sents[0:5]

['Adolescente de 13 anos com DW com confirmação genética e apresentação neurológica por doença de movimento (parkinsonismo e distonia) com agravamento progressivo, sem evidência clínica ou analítica de atingimento hepático.',
 'A biópsia hepática apresentava infiltrado inflamatório e sinais de hepatopatia crónica, sem cirrose.',
 'A RMN cerebral mostrou hipersinal T2 nos gânglios da base (caudado e putámen), sem alterações em T1 e restrição à difusão no caudado, núcleo lenticular e mesencéfalo.',
 'O início do tratamento com penicilamina despoletou uma progressiva deterioração neurológica sem estabilização após 11 meses de follow-up.']

##### Tokenização
Serve para separar o texto em *tokens* - que são uma sequência de caracteres com algum significado semântico.

**Principais dificuldades**:
*   "São Paulo" - uma ou duas palavras?
*   "A seleção dos E.U.A. venceu." - Pontuação pode ser considerada quebra de sentença
*   "Tromba d'água"
*   "Interação humano-computador"

In [ ]:
# Exemplo de tokenização a nivel de palavra
texto = sents[2]
# Tokeniza o texto
tokens = tokenize.word_tokenize(texto, language='portuguese')
print(tokens)

['A', 'RMN', 'cerebral', 'mostrou', 'hipersinal', 'T2', 'nos', 'gânglios', 'da', 'base', '(', 'caudado', 'e', 'putámen', ')', ',', 'sem', 'alterações', 'em', 'T1', 'e', 'restrição', 'à', 'difusão', 'no', 'caudado', ',', 'núcleo', 'lenticular', 'e', 'mesencéfalo', '.']


Quantos caracteres temos?

In [ ]:
len(texto)

166

Quantos tokens temos?

In [ ]:
len(tokens)

32

Quantos tokens únicos nós temos?

In [ ]:
len(set(tokens))

28

In [ ]:
# Usando a biblioteca collections
from collections import Counter

contador = Counter(tokens)

for cont in contador.items():
  print(cont)

('A', 1)
('RMN', 1)
('cerebral', 1)
('mostrou', 1)
('hipersinal', 1)
('T2', 1)
('nos', 1)
('gânglios', 1)
('da', 1)
('base', 1)
('(', 1)
('caudado', 2)
('e', 3)
('putámen', 1)
(')', 1)
(',', 2)
('sem', 1)
('alterações', 1)
('em', 1)
('T1', 1)
('restrição', 1)
('à', 1)
('difusão', 1)
('no', 1)
('núcleo', 1)
('lenticular', 1)
('mesencéfalo', 1)
('.', 1)


In [ ]:
# Mostra os termos mais frequentes
contador.most_common(2)

[('e', 3), ('caudado', 2)]

##### *Stemming*
Reduz as palavras ao seu *stem*, retirando o sufixo. Faz com que palavras de mesmo significado semântico (ou similar) sejam escritas da mesma maneira (e.g., correr, correndo, correu). Geralmente o *stem* não é uma palavra válida.

Para conhecer mais: https://www.nltk.org/_modules/nltk/stem/rslp.html


In [ ]:
# Caso não tenha feito o download de todos recursos do NLTK, você pode fazê-lo de maneira individual
# obtendo o stemmer para português (cada idioma tem suas regras próprias)
nltk.download('rslp')

# Inicia o Stemmer
stemmer = nltk.stem.RSLPStemmer()

print(stemmer.stem("ferro"))
print(stemmer.stem("ferreiro"))

print(stemmer.stem("correr"))
print(stemmer.stem("correu"))

ferr
ferr
corr
corr


[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [ ]:
# Define uma função que faz Stemming em todo um texto
def Stemming(texto):
  stemmer = nltk.stem.RSLPStemmer()
  novotexto = []
  for token in texto:
    novotexto.append(stemmer.stem(token.lower()))
  return novotexto

texto1 = "Eu gostei de correr"
texto2 = "Eu gosto de corrida"

# Tokeniza o texto
tokens1 = tokenize.word_tokenize(texto1, language='portuguese')
tokens2 = tokenize.word_tokenize(texto2, language='portuguese')

novotexto1 = Stemming(tokens1)
novotexto2 = Stemming(tokens2)

print(novotexto1)
print(novotexto2)

['eu', 'gost', 'de', 'corr']
['eu', 'gost', 'de', 'corr']


In [ ]:
# aplicando no nosso texto
tokens = tokenize.word_tokenize(texto, language='portuguese')
novotexto = Stemming(tokens)

print("Texto original:")
print(texto)
print("\nTexto com steming:")
print(' '.join(novotexto))

Texto original:
A RMN cerebral mostrou hipersinal T2 nos gânglios da base (caudado e putámen), sem alterações em T1 e restrição à difusão no caudado, núcleo lenticular e mesencéfalo.

Texto com steming:
a rmn cerebr mostr hipers t2 no gângli da bas ( caud e putámen ) , sem alter em t1 e restr à difus no caud , núcle lenticul e mesencéfal .


##### Lematização
Similar ao processo de *Stemming*, porém, faz uma análise morfológica completa para identificar e remover os sufixos. Geralmente leva os verbos ao infinitivo e substantivos/adjetivos ao masculino singular. Se diferencia do *Stemming* pois sempre gera uma palavra válida.

Infelizmente, esta funcionalidade não é suportada pelo NLTK. [Neste link](https://lars76.github.io/nlp/lemmatize-portuguese/) você pode encontrar alternativas para realizar a lematização em português.


##### Retirada de *Stop-words*
As vezes é necessário remover as palavras de maior ocorrência no conjunto de textos, pois geralmente elas não agregam grande valor semântico aos textos e não ajudam no processo de selecionar as informações relevantes ao sistema de PLN.

Este processo pode ser diferente, de acordo com a tarefa de PLN que você está executando, mas no geral temos duas abordagens: retirar as palavras de maior ocorrência levando em conta a [lei de Zipf](http://terrierteam.dcs.gla.ac.uk/publications/rtlo_DIRpaper.pdf), ou utilizar uma lista de *stop-words* pronta para seu idioma. Iremos realizar a segunda opção.

In [ ]:
# Caso não tenha feito o download de todos recursos do NLTK, você pode fazê-lo de maneira individual
nltk.download('stopwords')

# O NLTK fornece uma lista de stop-words para o idioma português
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords[:20]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele']

In [ ]:
# Define uma função que remove as stop words de um texto
def removeStopWords(texto):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    novotexto = []
    for token in texto:
        if token.lower() not in stopwords:
            novotexto.append(token)
    return novotexto

# envia o texto tokenizado
novotexto = removeStopWords(tokens)

print("Texto original:")
print(texto)
print("\nTexto sem stop words:")
print(' '.join(novotexto))

Texto original:
A RMN cerebral mostrou hipersinal T2 nos gânglios da base (caudado e putámen), sem alterações em T1 e restrição à difusão no caudado, núcleo lenticular e mesencéfalo.

Texto sem stop words:
RMN cerebral mostrou hipersinal T2 gânglios base ( caudado putámen ) , alterações T1 restrição difusão caudado , núcleo lenticular mesencéfalo .


> **IMPORTANTE**: Em alguns casos retirar as palavras referentes a negação (i.e., não) pode retirar um significado semântico muito importante do texto. Por exemplo, no texto: "*O paciente não apresenta sintomas da doença*". Neste caso a negação muda completamente o sentido da frase. Existem alguns outros casos (principalmente quando utilizamos *Deep Learning*) em que **a retirada das stop-words pode ser prejudicial ao algoritmo**, portanto, sempre teste seus algoritmos com e sem esta opção!

##### Normalização
Além do *Stemming*, é possível realizar processos mais específicos de normalização do texto, de acordo com a tarefa.

> **Exemplo 1**: O texto bruto pode estar escrito como "O preço inicial é de 23 reais, podendo chegar a mil". Em determinadas situações, é mais interessante que o texto seja normalizado para: "O preço inicial é de vinte e três reais, podendo chegar a mil"

> **Exemplo 2**: Todas as datas serem normalizadas para um padrão único. As formas "18/mai", "dezoito de maio", "18-05" serem normalizadas para "18/05".

> **Exemplo 3**: Todas diferentes menções a uma mesma doença em um texto serem representadas por um texto único. Por exemplo, "HAS", "H.A.S.", "Hipertensão", "Hipertensão Arterial", "hipertensão arterial sistêmica", "paciente hipertensivo".

Uma maneira simples de normalizar o texto é transformar todas as letras em minúsculas, assim ocorrências escritas de maneira diferente são normalizadas para uma única. "Brasil", "BraSil" e "BRAsil" seriam normalizadas para "brasil".

In [ ]:
texto2 = "Se escreve LOL, LoL ou Lol?"

# Efetua lowercase
texto2 = texto2.lower()

# Tokeniza o texto
tokenize.word_tokenize(texto2, language='portuguese')

['se', 'escreve', 'lol', ',', 'lol', 'ou', 'lol', '?']

### **2) Reconhecimeno de entidades nomeadas (NER)**

O **NER** (do inglês "*Named Entity Recognition*") é uma subtarefa de PLN que busca localizar e classificar entidades nomeadas mencionadas em texto não estruturado em categorias predefinidas, como nomes de pessoas, organizações, locais, códigos médicos, expressões de tempo, quantidades, valores monetários, porcentagens, etc.

A finalidade do NER é extrair informações relevantes do texto e atribuir um rótulo a cada entidade encontrada. Essa tarefa é útil em diversas aplicações, como análise de sentimentos, resumo automático de textos, tradução automática, extração de informações em documentos, entre outras.

Na área clinica, por exemplo, o NER pode ser usado para identificar condições médicas em documentos clínicos, como relatórios de exames, históricos médicos e resumos de alta hospitalar, auxiliando na triagem de pacientes, na detecção precoce de doenças e na pesquisa clínica.

Para o NER em textos clínicos, vamos usar um modelo treinado pelo nosso grupo: `pucpr/clinicalnerpt-disorder`, para extrair entidades relacionadas a **desordens** no paciente.

Neste exemplo, vamos utilizar *Machine Learning* (ML) para o reconhecimento das entidades nomeadas, mais especificamente *deep learning* e arquitetura *Transformer*.

#### **O que é *Machine Learning*?**

É uma área da Inteligência Artificial que provê a sistemas a habilidade de **automaticamente aprender tarefas sem que seja explicitamente programada para tal**.

> **OBSERVAÇÃO**: Atualmente muitos dos modelos estado da arte fazem uso dos [Transformers](https://huggingface.co/transformers/), que disponibilizam uma série de arquiteturas para processamento de textos.

##### **Executando a tarefa de NER**
Vamos utilizar um modelo de NER treinado pelo nosso grupo HAILab, usando arquitetura Transformer: `pucpr-br/postagger-bio-portuguese`

> **OBSERVAÇÃO**: Para mais modelos disponibilizados pelo nosso grupo, acesse: https://huggingface.co/pucpr e https://huggingface.co/pucpr-br.

Primeiro, precisamos instalar a biblioteca `transformers`, do grupo *Hugging Faces*, para importar o modelo e executar as tarefas.  

Vamos usar `pipeline`, objetos da biblioteca que abstraem a complexidade do código e oferecem uma API para várias tarefas, como NER, análise de sentimentos, etc. Para saber mais sobre pipelines: https://www.kaggle.com/code/funtowiczmo/hugging-face-transformers-how-to-use-pipelines

In [ ]:
!pip3 install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.4 MB/s eta 0:00:00


In [ ]:
# importanto as funcoes necessarias
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# importanto o tokenizador e o modelo de NER
model = AutoModelForTokenClassification.from_pretrained("pucpr/clinicalnerpt-disorder")
tokenizer = AutoTokenizer.from_pretrained('pucpr/clinicalnerpt-disorder')

nlp_token_class = pipeline('token-classification', model=model, tokenizer=tokenizer, grouped_entities=True)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [ ]:
textos[:2]

['Adolescente de 13 anos com DW com confirmação genética e apresentação neurológica por doença de movimento (parkinsonismo e distonia) com agravamento progressivo, sem evidência clínica ou analítica de atingimento hepático. A biópsia hepática apresentava infiltrado inflamatório e sinais de hepatopatia crónica, sem cirrose. A RMN cerebral mostrou hipersinal T2 nos gânglios da base (caudado e putámen), sem alterações em T1 e restrição à difusão no caudado, núcleo lenticular e mesencéfalo. O início do tratamento com penicilamina despoletou uma progressiva deterioração neurológica sem estabilização após 11 meses de follow-up.',
 'Homem de 65 anos, com antecedentes de diabetes mellitus tipo 2 controlada, com quadro de 3 meses de evolução caracterizado por tosse não produtiva, mialgias, febre, anorexia e perda de peso. Apresenta posteriormente agravamento com quadro de confusão mental com 4 dias de evolução, razão pela qual recorre ao Serviço de Urgência. À observação, febril (38.5ºC), lenti

In [ ]:
resultadoNER = nlp_token_class(textos[0])
resultadoNER

[{'entity_group': 'Disorder',
  'score': 0.91563916,
  'word': 'd',
  'start': 27,
  'end': 28},
 {'entity_group': 'Disorder',
  'score': 0.8767578,
  'word': '##w',
  'start': 28,
  'end': 29},
 {'entity_group': 'Disorder',
  'score': 0.99879646,
  'word': 'doen',
  'start': 86,
  'end': 90},
 {'entity_group': 'Disorder',
  'score': 0.9971208,
  'word': '##ca de movimento',
  'start': 90,
  'end': 105},
 {'entity_group': 'Disorder',
  'score': 0.9470548,
  'word': 'park',
  'start': 107,
  'end': 111},
 {'entity_group': 'Disorder',
  'score': 0.9611911,
  'word': '##ins',
  'start': 111,
  'end': 114},
 {'entity_group': 'Disorder',
  'score': 0.97013456,
  'word': '##onis',
  'start': 114,
  'end': 118},
 {'entity_group': 'Disorder',
  'score': 0.96701217,
  'word': '##mo',
  'start': 118,
  'end': 120},
 {'entity_group': 'Disorder',
  'score': 0.9953015,
  'word': 'dis',
  'start': 123,
  'end': 126},
 {'entity_group': 'Disorder',
  'score': 0.9963064,
  'word': '##toni',
  'start': 

##### Formatando a saida
Como o nosso tokenizador (*WordPiece Tokenizer*) divide as palavras em *sub-tokens* pelo seu significado básico + complemento, adicionando "##" no inicio da sub-palavra, vamos criar uma função para juntar as palavras novamente.

In [ ]:
def get_entities(ner_output):
    entities = []
    current_entity = ""

    for i, item in enumerate(ner_output):
        entity_word = item['word']
        if entity_word.startswith("##"):
          continue
        current_entity = entity_word
        next_word_index = i + 1

        for j in range(i, len(ner_output)-1):
          next_word_index2 = j + 1
          if ner_output[next_word_index2]['word'].startswith("##"):
            current_entity += ner_output[next_word_index2]['word'][2:]
          else:
            if current_entity:
                  entities.append(current_entity)
            break

    if current_entity:
      entities.append(current_entity)
      current_entity = ""

    return entities

In [ ]:
entity_words = get_entities(resultadoNER)
print("Entidades do tipo DISORDER encontradas no texto:")
print(entity_words)

Entidades do tipo DISORDER encontradas no texto:
['dw', 'doenca de movimento', 'parkinsonismo', 'distonia', 'agravamento progressivo', 'atingimento hepatico', 'hepatopatia cronica', 'cirrose', 'neurologica']


> **Por que não utilizar um modelo generalista, treinado por ex em textos jornalísticos?**
Por mais que os modelos generalistas tenham sido treinados com volumes maiores de dados, eles têm diversas características diferentes dos textos clínicos, bem como, um vocabulário diferente, fazendo com que o desempenho do modelo seja muito baixo em textos especializados em algum assunto, como é o texto clínico.

> **OBSERVAÇÃO**: Você poderia optar pela utilização de outras ferramentas de *Machine Learning* para usar ou mesmo treinar um modelo de NER, como o [scikit-learn](https://scikit-learn.org/stable/) e [Flair](https://github.com/flairNLP/flair).

### **3) Explorando um modelo Word2vec treinado com *twittes* brasileiros sobre Covid-19**
Agora vamos explorar um modelo Word2Vec treinado pelo nosso grupo, usando *twittes* brasileiros sobre a Covid-19. Para ler mais sobre o trabalho, acesse: https://jhi.sbis.org.br/index.php/jhi-sbis/article/view/842

Para ter acesso ao código com mais exemplos, acesse: https://github.com/HAILab-PUCPR/Word2Vec-COVID19-Twitter

![picture](https://github.com/HAILab-PUCPR/Word2Vec-COVID19-Twitter/blob/master/nuvem-tags.jpg?raw=true)



#### Baixando o modelo W2V



In [ ]:
!curl -L -o 'word2vec_tweets.bin' 'https://drive.google.com/uc?export=download&id=1VEL--MvQW49WDaf1_m3K7D9M60FD_N8D'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:04 --:--:--     0
100 92.6M  100 92.6M    0     0  16.5M      0  0:00:05  0:00:05 --:--:--  131M


In [ ]:
!ls

PortugueseClinicalNER  sample_data  word2vec_tweets.bin


Vamos utilizar a biblioteca `gensim`, amplamente utilizada para modelagem de tópicos, similaridade entre documentos, transformação de texto e modelos de incorporação de palavras.

O `Gensim` suporta a criação e o treinamento de modelos de incorporação de palavras, como o Word2Vec e o Doc2Vec, que podem ser usadas para várias tarefas de PLN, como análise de sentimento, classificação de documentos e identificação de palavras similares.


In [ ]:
from gensim.models import KeyedVectors

# Carregar o modelo Word2Vec binário
model = KeyedVectors.load_word2vec_format("word2vec_tweets.bin", binary=True)


Buscando palavras similares

In [ ]:
model.most_similar(positive=['covid'], topn=20)


[('coronavirus', 0.8442822694778442),
 ('Covid', 0.7996824979782104),
 ('covid.', 0.7172111868858337),
 ('cov', 0.7085482478141785),
 ('COVID', 0.6961955428123474),
 ('corona_virus', 0.6780412793159485),
 ('corona', 0.6579604148864746),
 ('Coronavirus', 0.6548892259597778),
 ('covid...', 0.6536855697631836),
 ('covi', 0.6502013206481934),
 ('Corona_Virus', 0.6478165984153748),
 ('coronavirus...', 0.6453582048416138),
 ('corona_virus.', 0.6387130618095398),
 ('Covid.', 0.6371161937713623),
 ('coronavirus.', 0.6351439356803894),
 ('covid?', 0.634706974029541),
 ('Corona_virus', 0.6235688924789429),
 ('coron', 0.5937241315841675),
 ('coro', 0.587263822555542),
 ('doenca', 0.5840329527854919)]

In [ ]:
model.most_similar(positive=['enxaqueca'], topn=20)


[('ansiedade', 0.5865851044654846),
 ('alergia', 0.55230313539505),
 ('crise_alergica', 0.5322316288948059),
 ('rinite', 0.5309249758720398),
 ('sinusite', 0.5242369771003723),
 ('colica', 0.5206822156906128),
 ('dor', 0.4971705973148346),
 ('ansiedade.', 0.49190571904182434),
 ('gastrite', 0.4848136305809021),
 ('amigdalite', 0.4826139211654663),
 ('renite', 0.47357383370399475),
 ('refluxo', 0.46765759587287903),
 ('febre', 0.4607495367527008),
 ('insonia', 0.4539395272731781),
 ('tpm', 0.4536483585834503),
 ('asma', 0.4515649676322937),
 ('tosse_seca', 0.45089566707611084),
 ('crise_existencial', 0.4400123357772827),
 ('asmatica', 0.43962305784225464),
 ('alergica', 0.4376436769962311)]

In [ ]:
model.most_similar(positive=['vacina'], topn=20)


[('vacina_contra', 0.8123534917831421),
 ('vacina_pro', 0.7428391575813293),
 ('vacina.', 0.6893975734710693),
 ('cura', 0.6615239977836609),
 ('Vacina', 0.6016098856925964),
 ('vacina?', 0.5801373720169067),
 ('cura_pro', 0.5537213683128357),
 ('vacinas', 0.5207374095916748),
 ('vac', 0.5135458111763),
 ('vacin', 0.4958423376083374),
 ('vacina...', 0.49168628454208374),
 ('vaci', 0.483140230178833),
 ('vacina_chinesa', 0.4786967933177948),
 ('Vacina_contra', 0.47071388363838196),
 ('remedio', 0.4576752185821533),
 ('vacina!', 0.44519132375717163),
 ('anticorpos', 0.4302246868610382),
 ('anticorpo', 0.42934349179267883),
 ('anticorpo_eficaz', 0.4280613660812378),
 ('medicacao', 0.42788663506507874)]

In [ ]:
model.most_similar(positive=['cloroquina'], topn=20)


[('Cloroquina', 0.9237001538276672),
 ('hidroxicloroquina', 0.8887606263160706),
 ('Hidroxicloroquina', 0.8367518782615662),
 ('cloroquina.', 0.7896125912666321),
 ('cloroquina?', 0.7413284778594971),
 ('cloroquina!', 0.7372238636016846),
 ('cloroquina...', 0.7071815133094788),
 ('Cloroquina.', 0.7057933807373047),
 ('hidroxicloroquina.', 0.7038278579711914),
 ('remedio', 0.6733832955360413),
 ('cloroqui', 0.6718700528144836),
 ('HCQ', 0.6606883406639099),
 ('CLOROQUINA', 0.6528093814849854),
 ('cloroqu', 0.6511754393577576),
 ('Hidroxicloroquina.', 0.6503738760948181),
 ('Cloroquina?', 0.6487926244735718),
 ('cloroquin', 0.6407572031021118),
 ('medicacao', 0.6388699412345886),
 ('clor', 0.6387254595756531),
 ('azitromicina', 0.6380695700645447)]

In [ ]:
model.most_similar(positive=['bolsonaro'], topn=20)


[('bozo', 0.8736352324485779),
 ('biroliro', 0.7642959356307983),
 ('Bozo', 0.75406414270401),
 ('Bolsonaro', 0.7218829989433289),
 ('bonoro', 0.6744195222854614),
 ('presidente', 0.6403055787086487),
 ('ele', 0.6385378241539001),
 ('bozonaro', 0.6303306221961975),
 ('Biroliro', 0.6025436520576477),
 ('nosso_presidente', 0.6010631322860718),
 ('bolsolixo', 0.585766077041626),
 ('salnorabo', 0.5816051363945007),
 ('esse_homem', 0.5811737179756165),
 ('trump', 0.5701310634613037),
 ('bostonaro', 0.5657585263252258),
 ('bolsonaro.', 0.549957811832428),
 ('mito', 0.5496857762336731),
 ('cara', 0.5403366088867188),
 ('miliciano', 0.5294407606124878),
 ('vc', 0.5216509699821472)]

In [ ]:
model.most_similar(positive=['hipertensao'], topn=20)


[('diabetes', 0.5345937013626099),
 ('cardiopatia', 0.526973307132721),
 ('diabetes_hipertensao', 0.4997483193874359),
 ('problema_cardiaco', 0.49578699469566345),
 ('hipertensa', 0.48136430978775024),
 ('hipertenso', 0.47349655628204346),
 ('psoriase', 0.4651118814945221),
 ('pressao_alta', 0.4613089859485626),
 ('doenca_cronica', 0.4564073085784912),
 ('comorbidade', 0.4466204047203064),
 ('problemas_cardiacos', 0.44344663619995117),
 ('hipertensos', 0.44098711013793945),
 ('sobrepeso', 0.43912020325660706),
 ('diabete', 0.4285372197628021),
 ('diabetico', 0.42610156536102295),
 ('DPOC', 0.4237419366836548),
 ('idade_avancada', 0.4208346903324127),
 ('cancer', 0.41892048716545105),
 ('diabetica', 0.41527238488197327),
 ('enfisema', 0.41280341148376465)]

In [ ]:
model.most_similar(positive=['diabetes'], topn=20)


[('diabete', 0.572637677192688),
 ('hipertensao', 0.5345937609672546),
 ('diabetes_hipertensao', 0.5322251319885254),
 ('sobrepeso', 0.5195519924163818),
 ('comorbidade', 0.5192655920982361),
 ('cardiopatia', 0.5159457921981812),
 ('DPOC', 0.5149514079093933),
 ('pressao_alta', 0.5084439516067505),
 ('alguma_comorbidade', 0.4888414442539215),
 ('cancer', 0.4604306221008301),
 ('trombose', 0.4535661041736603),
 ('Alzheimer', 0.44834133982658386),
 ('obesidade', 0.4459187090396881),
 ('comorbidades', 0.4449557662010193),
 ('problema_cardiaco', 0.44046324491500854),
 ('diabetica', 0.43660256266593933),
 ('Diabetes', 0.43450477719306946),
 ('doenca_cronica', 0.4337991774082184),
 ('infarto', 0.43118056654930115),
 ('problemas_respiratorios', 0.428737074136734)]

In [ ]:
model.similarity("doenca", 'dor')


0.24872306

In [ ]:
model.similarity("virus", 'bacteria')


0.39860615

In [ ]:
model.similarity("virus", 'agua')


0.07849422

In [ ]:
model.doesnt_match(['covid', 'corona', 'oi'])


'oi'

### Analogias
Qual palavra está para rainha assim como homem está para rei?

In [ ]:
similar_words = model.most_similar(positive=["rainha", "homem"], negative=["rei"], topn=1)
similar_word = similar_words[0][0]

print(similar_word)

mulher


In [ ]:
# Importante! Erros podem acontecer

similar_words = model.most_similar(positive=["madrid", "franca"], negative=["paris"], topn=1)
similar_word = similar_words[0][0]

print(similar_word)

italia


### **Conclusão**
Neste breve tutorial repassamos pelos conceitos básicos de PLN, como operaçõs de pré-processamento de texto, tarefa de **Named Entity Recognition** e **Word2Vec**.

Estes scripts podem ser ponto de partida para diversos **Sistemas de Apoio a Decisão em Saúde** que fazem uso de informações contidas em textos clínicos.

Você pode encontrar informações adicionais sobre PLN clínico nos *links* a seguir.

*   [Language Processing Tools Improve Care Delivery for Providers](https://healthtechmagazine.net/article/2020/05/language-processing-tools-improve-care-delivery-providers)
*   [Natural language processing in healthcare](https://www.mckinsey.com/industries/healthcare-systems-and-services/our-insights/natural-language-processing-in-healthcare#)
*   [Lessons learned building natural language processing systems in health care](https://www.oreilly.com/content/lessons-learned-building-natural-language-processing-systems-in-health-care/)
*   [Introduction to Clinical Natural Language Processing: Predicting Hospital Readmission with Discharge Summaries](https://towardsdatascience.com/introduction-to-clinical-natural-language-processing-predicting-hospital-readmission-with-1736d52bc709)
* [List of resources and tools developed with focus on Portuguese](https://github.com/ajdavidl/Portuguese-NLP)


Este notebook foi produzido por [Elisa Terumi](https://www.linkedin.com/in/elisa-terumi-rubel-schneider/) , tendo como base o material do Prof. Dr. [Lucas Oliveira](http://lattes.cnpq.br/3611246009892500).